# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
r = requests.get(url)
data_json = r.json()
df_rj = pd.DataFrame(data_json['items'])

In [3]:
# 2) Seu código aqui
df_rj['status'].value_counts().head(3)

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: status, dtype: int64

In [4]:
# 3) Seu código aqui
list_ano = []

for i in df_rj['data_status'].str.split('/'):
    list_ano.append(i[-1])

df_rj['ano_status'] = list_ano

In [5]:
## OUTRA FORMA

list_ano = []

for i in df_rj['data_status'].str.slice():
    list_ano.append(i[-4:])

df_rj['ano_status'] = list_ano

In [6]:
# 4) Seu código aqui
df_rj['ano_status'].value_counts()

2012    11
2009     9
2013     7
2011     6
2002     4
2014     4
2016     4
2010     3
2006     2
2008     2
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [7]:
#1) Seu código aqui
def return_df(uf, interessado):
    
    if interessado[:3].lower() in 'estado':
        interessado = 'Estado'
    elif interessado[:3].lower() in 'municipio':
        interessado = 'Município'
    
    url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=' + uf.upper() + '&tipo_interessado='+ interessado
    r = requests.get(url)
    data_json = r.json()
    
    return pd.DataFrame(data_json['items'])

In [8]:
# 2) Seu código aqui
df_mg = return_df('mg','estado')

df_mg[df_mg['status'] == 'Arquivado por decurso de prazo'].status.value_counts()

Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [9]:
# 3) Seu código aqui
df_ba = return_df('ba','municipio')

df_ba.loc[df_ba['status'] == 'Deferido', 'interessado'].value_counts().head(1)

Luís Eduardo Magalhães    14
Name: interessado, dtype: int64

In [10]:
#OUTRA FORMA
df_badeferido = df_ba[df_ba['status'] == 'Deferido']

df_badeferido['interessado'].value_counts().head(1)

Luís Eduardo Magalhães    14
Name: interessado, dtype: int64

In [11]:
# 4) Seu código aqui
return_df('ba','estado').to_csv('./bahia_estado.csv')